In [1]:
import random
import tomllib

import matplotlib.pyplot as plt
from numba import jit

from util import *

In [2]:
with open('parameters.toml', 'r') as f:
    parameters = tomllib.loads(f.read())
    
parameters

{'TRAINING': {'healthy_skin_path': './dataset/NORMAL SKIN/',
  'diseased_skin_path': './dataset/PSORIASIS/',
  'training_dataset_ratio': 0.8}}

In [3]:
# base_dataset = {'training': list(), 'validation': list(), 'testing': list()}

# raw_healthy_images = read_images(parameters['TRAINING']['healthy_skin_path'])
# raw_diseased_images = read_images(parameters['TRAINING']['diseased_skin_path'])

# total_raw_images = raw_healthy_images + raw_diseased_images

# random.shuffle(total_raw_images)

# for i, image in enumerate(total_raw_images):
#     training_ratio = parameters['TRAINING']['training_dataset_ratio']

#     if i < len(total_raw_images) * training_ratio:
#         base_dataset['training'].append(image)
#     elif i < len(total_raw_images) * round((1 + training_ratio) / 2, 1):
#         base_dataset['validation'].append(image)
#     else:
#         base_dataset['testing'].append(image)
        
# print(f"Total dataset: {len(total_raw_images)} items.\n")
# print(f"Training: {len(base_dataset['training'])} items.")
# print(f"Validation: {len(base_dataset['validation'])} items.")
# print(f"Testing: {len(base_dataset['testing'])} items.")